# Github - Get list of active projects
The objective of the notebook is provide a list of active projects that an organization is working on

**Tags:** #naas #github #projects


## Input

### Imports

In [1]:
import requests
import pandas as pd
from urllib.parse import urlencode
from datetime import datetime

### Variables

In [2]:
PROJECT_URL = "https://github.com/orgs/jupyter-naas/projects"
GITHUB_TOKEN = "ghp_CEvqR7QauDbNLRiIiwACxxxxxxx"

## Model

In [3]:
def get_active_project_links(token, url):
    project_df = pd.DataFrame()
    headers={"Authorization":f'token {token}'}
    url = "api.github.com".join(url.split("github.com"))
    page=1
    while True:
        params = {
            "per_page":100,
            'page':page
        }
        url_link = url+ f"?{urlencode(params, safe='(),')}"
        res = requests.get(url_link, headers=headers, params=params)
        
        try:
            res.raise_for_status()
        except requests.HTTPError as e:
            raise(e)
        if len(res.json())==0:
            break
        res_json = res.json()

        for idx, project in enumerate(res_json):
            project_df.loc[idx, 'project_name'] = project.get('name')
            project_df.loc[idx, 'project_description'] = project.get('body')
            project_df.loc[idx, 'project_id'] = project.get('number')
            project_df.loc[idx, 'project_created_by'] = project.get('creator')['login']
            
            project_df.loc[idx, 'project_created_date'] = project.get('created_at').strip('Z').split('T')[0]
            project_df.loc[idx, 'project_created_time'] = project.get('created_at').strip('Z').split('T')[-1]
            project_df.loc[idx, 'project_updated_date'] = project.get('updated_at').strip('Z').split('T')[0]
            project_df.loc[idx, 'project_updated_time'] = project.get('updated_at').strip('Z').split('T')[-1]
            
            project_df.loc[idx, 'project_columns_url'] = project.get('columns_url')
            
        page+=1
        
    project_df['project_id'] = project_df.project_id.astype('int')
    return project_df

df_projects = get_active_project_links(GITHUB_TOKEN, PROJECT_URL)

## Output

In [4]:
df_projects.head()

,project_name,project_description,project_id,project_created_by,project_created_date,project_created_time,project_updated_date,project_updated_time,project_columns_url
0,Community Roadmap,Official Naas Community Roadmap,4,jravenel,2021-05-25,20:53:14,2022-02-11,05:58:23,https://api.github.com/projects/12537896/columns
